In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set Chrome options to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU for better performance
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource issues

In [3]:
def find_team(text):
# Function to check matches in a given list
    def check_list(team_list):
        matches = [word for word in team_list if word in text]
        if len(matches) > 1:
            raise ValueError(f"Error: More than one match found: {matches}")
        elif len(matches) == 1:
            return matches[0]
        return None

    team_names_sf = ["KKR","GT","MI","CSK","RR","RCB","PBKS","DC","SRH","LSG"]
    team_names_ff = ["Kolkata Knight Riders", "Gujarat Titans", "Mumbai Indians", "Chennai Super Kings","Rajasthan Royals","Royal Challengers Bangalore", "Punjab Kings","Delhi Capitals","Sunrisers Hyderabad","Lucknow Super Giants"]

    # Check in first list
    match = check_list(team_names_sf)
    if match:
        return team_names_ff[team_names_sf.index(match)]
    
    # Check in second list if no match found in first list
    match = check_list(team_names_ff)
    if match:
        return match

    raise ValueError("Error: No match found in either list")

In [27]:
def scorecard(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Initialize lists to store data for the match
    innings_list = []
    batsmen_list = pd.DataFrame()
    match_score = pd.DataFrame()
    did_not_bat = pd.DataFrame()
    bowlers_info = pd.DataFrame()

    winner = soup.find('p',class_="ds-text-tight-s ds-font-medium ds-truncate ds-text-typo").text.strip()
    winner = find_team(winner)

    match_details_table = soup.find('table',class_= "ds-w-full ds-table ds-table-sm ds-table-auto") 
    table_rows = match_details_table.find_all('tr')
    for row in table_rows:
        if "Player Of The Match" in row.text:
            man_of_the_match = row.find('td',class_="ds-text-typo").text.strip()
    
    # Find all divs with class "ds-rounded-lg ds-mt-2" (which contain innings tables)
    innings_tables = soup.find_all('div', class_='ds-rounded-lg ds-mt-2')

    # Loop through each innings table div
    for innings_table in innings_tables:

        innings_number = innings_tables.index(innings_table) + 1

        # Find the div that contains the team name and innings number
        team_innings_div = innings_table.find('div', class_='ds-flex ds-px-4 ds-border-b ds-border-line ds-py-3 ds-bg-ui-fill-translucent-hover')
        batting_innings = team_innings_div.text.strip().replace('\xa0',' ')
        batting_innings = find_team(batting_innings)
            
        innings_list.append(batting_innings)

        
        batsmen_table = innings_table.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table')
    
        # Batsmen details extraction
        batting_info = batsmen_table.find_all('tr', class_='')
        batsmen_stats = batting_info[:len(batting_info)-2]

        for batter in batsmen_stats:
            batsman_stats = batter.find_all('td')
            name = batsman_stats[0].text.strip()
            dismissal = batsman_stats[1].text.strip()
            runs = batsman_stats[2].text.strip()
            balls = batsman_stats[3].text.strip()
            minutes = batsman_stats[4].text.strip()
            fours = batsman_stats[5].text.strip()
            sixes = batsman_stats[6].text.strip()
            strike_rate = batsman_stats[7].text.strip()
                    
            # Append the details to the list
            batsmen_stat = {'Innings Number': innings_number,
                'Innings Name': batting_innings,
                'Batsman': name,
                'Dismissal': dismissal,
                'Runs': runs,
                'Balls': balls,
                'Minutes': minutes,
                '4s': fours,
                '6s': sixes,
                'Strike Rate': strike_rate
            }
            batsmen_list = batsmen_list._append(batsmen_stat, ignore_index = True)
        
        # Extract the total score, overs, and run rate
        total_row = batting_info[-2]  # The last 'tr' containing score info
        total_datapoints = total_row.find_all('td')
        if total_datapoints[0].text.strip() == "Total":
            score = total_datapoints[2].text.strip()
            other_info = total_datapoints[1].text.strip()
            overs = other_info.split(' Ov')[0]
            remaining = other_info.split('RR: ')[1].split(')')[0]
            if "Mins" not in remaining:
                run_rate = remaining
                mins = "-"
            else:
                run_rate = remaining.split(', ')[0]
                mins = remaining.split(', ')[1].split(" Mins")[0]                   
            # Add score, overs, and run rate to the match score list
            innings_score = {'Innings Number': innings_number,
                'Innings Name': batting_innings,
                'Score': score,
                'Overs': overs,
                'Run Rate': run_rate,
                "Minutes" : mins
            }
            match_score = match_score._append(innings_score, ignore_index = True)

        # Extract "Did Not Bat" information
        did_not_bat_row = batsmen_table.find('tr', class_='!ds-border-b-0')
        if did_not_bat_row:
            batsmen = did_not_bat_row.find_all('div', class_='ds-popper-wrapper ds-inline')
            for batsman in batsmen:
                batsman_name = (batsman.get_text(strip=True)).replace(",", "")
                did_not_bat = did_not_bat._append({'Innings Number': innings_number, 'Innings Name': batting_innings, 'Batsman':batsman_name}, ignore_index = True)

                
        #Extracting Bowling Info
        bowling_table = innings_table.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto')

        # Check if the bowling table is present
        if bowling_table:
            headers = []
            head_data = bowling_table.find_all('th')
            for bowler_header in head_data:
                headers.append(bowler_header.text.strip())
            headers = list(map(lambda x: "Bowler" if x == "Bowling" else x, headers))
            headers = list(map(lambda x: "Overs" if x == "O" else x, headers))
            headers = list(map(lambda x: "Maidens" if x == "M" else x, headers))
            headers = list(map(lambda x: "Runs" if x == "R" else x, headers))
            headers = list(map(lambda x: "Wickets" if x == "W" else x, headers))
            headers = list(map(lambda x: "Economy" if x == "ECON" else x, headers))
            headers = list(map(lambda x: "Wides" if x == "WD" else x, headers))
            headers = list(map(lambda x: "No Balls" if x == "NB" else x, headers))
            
            bowler_rows = bowling_table.find('tbody').find_all('tr',class_='')

            for row in bowler_rows:
                bowler_data = {}
                bowler_data['Innings Number'] = innings_number
                bowler_data['Innings Name'] = batting_innings
                bowler_row_data = row.find_all('td')

                for i in range(len(headers)):
                    bowler_data[headers[i]] = bowler_row_data[i].text.strip()        

                # Append the bowler's data to the list
                bowlers_info = bowlers_info._append(bowler_data, ignore_index = True)
    
    #driver.quit()
    return winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info

In [28]:
def printing_scorecard(winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info):
    for innings in innings_list:
        print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")  
        print(innings + ":")
        print()
    
        print("Batsmen:")
        display(batsmen_list[batsmen_list['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
            {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
        ]))

        # Print Score Information
        print("Score:")
        display(match_score[match_score['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
            {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
        ]))
    
        # Print "Did Not Bat" information
        if did_not_bat.empty == False:
            if did_not_bat[did_not_bat['Innings Name'] == innings].empty == False:
                print("Did Not Bat:")
                display(did_not_bat[did_not_bat['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
                {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
            ]))
        
        print("Bowlers:")
        display(bowlers_info[bowlers_info['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
            {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
        ]))
    print("Winner: ",winner)
    print()
    print("Man of the Match: ",man_of_the_match)


In [29]:
#match_url = input("Enter the match url: ")
match_url = "https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-final-1426312/full-scorecard"

begin = time.time() 
winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info = scorecard(match_url)

printing_scorecard(winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info)
end = time.time()
# total time taken 
total_time_taken = end-begin
minutes = str(int(total_time_taken/60))
seconds = str(round(total_time_taken % 60,3))
total_time_taken = minutes+"m "+seconds+"s"
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")  
print(f"Total runtime of the program is {total_time_taken}")             

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Sunrisers Hyderabad:

Batsmen:


Batsman,Dismissal,Runs,Balls,Minutes,4s,6s,Strike Rate
Abhishek Sharma,b Starc,2,5,3,0,0,40.00
Travis Head,c †Rahmanullah Gurbaz b Arora,0,1,11,0,0,0.00
Rahul Tripathi,c Ramandeep Singh b Starc,9,13,21,1,0,69.23
Aiden Markram,c Starc b Russell,20,23,41,3,0,86.95
Nitish Kumar Reddy,c †Rahmanullah Gurbaz b Harshit Rana,13,10,14,1,1,130.00
Heinrich Klaasen †,b Harshit Rana,16,17,32,1,0,94.11
Shahbaz Ahmed,c Narine b Varun,8,7,5,0,1,114.28
Abdul Samad,c †Rahmanullah Gurbaz b Russell,4,4,3,0,0,100.00
Pat Cummins (c),c Starc b Russell,24,19,28,2,1,126.31
Jaydev Unadkat,lbw b Narine,4,11,16,0,0,36.36


Score:


Score,Overs,Run Rate,Minutes
113,18.3,6.10,-


Did Not Bat:


...

Did Not Bat:


Batsman
Rinku Singh
Andre Russell
Ramandeep Singh
Mitchell Starc
Vaibhav Arora
Harshit Rana
Varun Chakravarthy


Bowlers:


Bowler,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,Wides,No Balls
Bhuvneshwar Kumar,2,0,25,0,12.50,5,2,2,1,0
Pat Cummins,2,0,18,1,9.00,7,2,1,1,0
T Natarajan,2,0,29,0,14.50,2,4,1,2,0
Shahbaz Ahmed,2.3,0,22,1,8.80,7,1,2,0,0
Jaydev Unadkat,1,0,9,0,9.00,1,1,0,0,0
Aiden Markram,1,0,5,0,5.00,2,0,0,0,0


Winner:  Kolkata Knight Riders

Man of the Match:  Mitchell Starc
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Total runtime of the program is 0m 0.42s


In [20]:
print(innings_list)

['Sunrisers Hyderabad', 'Kolkata Knight Riders']


In [21]:
display(batsmen_list)
# batsmen_list.loc[batsmen_list['Innings Number'] == c]

,Innings Number,Innings Name,Batsman,Dismissal,Runs,Balls,Minutes,4s,6s,Strike Rate
0,1,Sunrisers Hyderabad,Abhishek Sharma,b Starc,2,5,3,0,0,40.00
1,1,Sunrisers Hyderabad,Travis Head,c †Rahmanullah Gurbaz b Arora,0,1,11,0,0,0.00
2,1,Sunrisers Hyderabad,Rahul Tripathi,c Ramandeep Singh b Starc,9,13,21,1,0,69.23
3,1,Sunrisers Hyderabad,Aiden Markram,c Starc b Russell,20,23,41,3,0,86.95
4,1,Sunrisers Hyderabad,Nitish Kumar Reddy,c †Rahmanullah Gurbaz b Harshit Rana,13,10,14,1,1,130.00
5,1,Sunrisers Hyderabad,Heinrich Klaasen †,b Harshit Rana,16,17,32,1,0,94.11
6,1,Sunrisers Hyderabad,Shahbaz Ahmed,c Narine b Varun,8,7,5,0,1,114.28
7,1,Sunrisers Hyderabad,Abdul Samad,c †Rahmanullah Gurbaz b Russell,4,4,3,0,0,100.00
8,1,Sunrisers Hyderabad,Pat Cummins (c),c Starc b Russell,24,19,28,2,1,126.31
9,1,Sunrisers Hyderabad,Jaydev Unadkat,lbw b Narine,4,11,16,0,0,36.36


In [22]:
display(match_score)

,Innings Number,Innings Name,Score,Overs,Run Rate,Minutes
0,1,Sunrisers Hyderabad,113,18.3,6.10,-
1,2,Kolkata Knight Riders,114/2,10.3,10.85,-


In [23]:
display(did_not_bat)

,Innings Number,Innings Name,Batsman
0,1,Sunrisers Hyderabad,T Natarajan
1,2,Kolkata Knight Riders,Rinku Singh
2,2,Kolkata Knight Riders,Andre Russell
3,2,Kolkata Knight Riders,Ramandeep Singh
4,2,Kolkata Knight Riders,Mitchell Starc
5,2,Kolkata Knight Riders,Vaibhav Arora
6,2,Kolkata Knight Riders,Harshit Rana
7,2,Kolkata Knight Riders,Varun Chakravarthy


In [24]:
display(bowlers_info)

,Innings Number,Innings Name,Bowler,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,Wides,No Balls
0,1,Sunrisers Hyderabad,Mitchell Starc,3,0,14,2,4.66,11,2,0,0,0
1,1,Sunrisers Hyderabad,Vaibhav Arora,3,0,24,1,8.00,9,2,1,4,0
2,1,Sunrisers Hyderabad,Harshit Rana,4,1,24,2,6.00,13,2,1,1,0
3,1,Sunrisers Hyderabad,Sunil Narine,4,0,16,1,4.00,9,0,0,0,0
4,1,Sunrisers Hyderabad,Andre Russell,2.3,0,19,3,7.60,7,1,1,1,0
5,1,Sunrisers Hyderabad,Varun Chakravarthy,2,0,9,1,4.50,7,1,0,0,0
6,2,Kolkata Knight Riders,Bhuvneshwar Kumar,2,0,25,0,12.50,5,2,2,1,0
7,2,Kolkata Knight Riders,Pat Cummins,2,0,18,1,9.00,7,2,1,1,0
8,2,Kolkata Knight Riders,T Natarajan,2,0,29,0,14.50,2,4,1,2,0
9,2,Kolkata Knight Riders,Shahbaz Ahmed,2.3,0,22,1,8.80,7,1,2,0,0


In [25]:
print(winner)

Kolkata Knight Riders


In [ ]:
def player_points(winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info):
    

SyntaxError: expected '(' (4285429750.py, line 1)

In [39]:
class Score:
    team_names_sf = ["KKR","GT","MI","CSK","RR","RCB","PBKS","DC","SRH","LSG"]
    team_names_ff = ["Kolkata Knight Riders", "Gujarat Titans", "Mumbai Indians", "Chennai Super Kings","Rajasthan Royals","Royal Challengers Bangalore", "Punjab Kings","Delhi Capitals","Sunrisers Hyderabad","Lucknow Super Giants"]

    def __init__(self,url):
        self.url = url
        self.winner,self.man_of_the_match,self.innings_list,self.batsmen_list,self.match_score,self.did_not_bat,self.bowlers_info = scorecard(self.url)

    def find_team(text):
        def check_list(team_list):
            matches = [word for word in team_list if word in text]
            if len(matches) > 1:
                raise ValueError(f"Error: More than one match found: {matches}")
            elif len(matches) == 1:
                return matches[0]
            return None

        match = check_list(team_names_sf)
        if match:
            return team_names_ff[team_names_sf.index(match)]
        match = check_list(team_names_ff)
        if match:
            return match
        raise ValueError("Error: No match found in either list")

    def scorecard(url):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        
        innings_list = []
        batsmen_list = pd.DataFrame()
        match_score = pd.DataFrame()
        did_not_bat = pd.DataFrame()
        bowlers_info = pd.DataFrame()

        winner = soup.find('p', class_="ds-text-tight-s ds-font-medium ds-truncate ds-text-typo").text.strip()
        winner = self.find_team(winner)

        match_details_table = soup.find('table', class_= "ds-w-full ds-table ds-table-sm ds-table-auto") 
        table_rows = match_details_table.find_all('tr')
        for row in table_rows:
            if "Player Of The Match" in row.text:
                man_of_the_match = row.find('td', class_="ds-text-typo").text.strip()
        
        innings_tables = soup.find_all('div', class_='ds-rounded-lg ds-mt-2')
        for innings_table in innings_tables:
            innings_number = innings_tables.index(innings_table) + 1
            team_innings_div = innings_table.find('div', class_='ds-flex ds-px-4 ds-border-b ds-border-line ds-py-3 ds-bg-ui-fill-translucent-hover')
            batting_innings = team_innings_div.text.strip().replace('\xa0',' ')
            batting_innings = self.find_team(batting_innings)
            innings_list.append(batting_innings)
            
            batsmen_table = innings_table.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table')
            batting_info = batsmen_table.find_all('tr', class_='')
            batsmen_stats = batting_info[:len(batting_info)-2]
            
            for batter in batsmen_stats:
                batsman_stats = batter.find_all('td')
                batsmen_stat = {
                    'Innings Number': innings_number,
                    'Innings Name': batting_innings,
                    'Batsman': batsman_stats[0].text.strip(),
                    'Dismissal': batsman_stats[1].text.strip(),
                    'Runs': batsman_stats[2].text.strip(),
                    'Balls': batsman_stats[3].text.strip(),
                    'Minutes': batsman_stats[4].text.strip(),
                    '4s': batsman_stats[5].text.strip(),
                    '6s': batsman_stats[6].text.strip(),
                    'Strike Rate': batsman_stats[7].text.strip()
                }
                batsmen_list = batsmen_list._append(batsmen_stat, ignore_index=True)
            
            total_row = batting_info[-2]
            total_datapoints = total_row.find_all('td')
            if total_datapoints[0].text.strip() == "Total":
                other_info = total_datapoints[1].text.strip()
                overs = other_info.split(' Ov')[0]
                run_rate = other_info.split('RR: ')[1].split(')')[0].split(', ')[0]
                innings_score = {
                    'Innings Number': innings_number,
                    'Innings Name': batting_innings,
                    'Score': total_datapoints[2].text.strip(),
                    'Overs': overs,
                    'Run Rate': run_rate
                }
                match_score = match_score._append(innings_score, ignore_index=True)
        
        return winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info
    
    def printing_scorecard(self):
        for innings in self.innings_list:
            print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")  
            print(innings + ":")
            print()
        
            print("Batsmen:")
            display(self.batsmen_list[self.batsmen_list['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
                {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
            ]))

            # Print Score Information
            print("Score:")
            display(self.match_score[self.match_score['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
                {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
            ]))
        
            # Print "Did Not Bat" information
            if self.did_not_bat.empty == False:
                if self.did_not_bat[self.did_not_bat['Innings Name'] == innings].empty == False:
                    print("Did Not Bat:")
                    display(self.did_not_bat[self.did_not_bat['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
                    {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
                    {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
                ]))
            
            print("Bowlers:")
            display(self.bowlers_info[self.bowlers_info['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
                {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
            ]))
        print("Winner: ",self.winner)
        print()
        print("Man of the Match: ",self.man_of_the_match)

In [40]:
url = "https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-final-1426312/full-scorecard"
match1 = Score(url)
match1.printing_scorecard()

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Sunrisers Hyderabad:

Batsmen:


Batsman,Dismissal,Runs,Balls,Minutes,4s,6s,Strike Rate
Abhishek Sharma,b Starc,2,5,3,0,0,40.00
Travis Head,c †Rahmanullah Gurbaz b Arora,0,1,11,0,0,0.00
Rahul Tripathi,c Ramandeep Singh b Starc,9,13,21,1,0,69.23
Aiden Markram,c Starc b Russell,20,23,41,3,0,86.95
Nitish Kumar Reddy,c †Rahmanullah Gurbaz b Harshit Rana,13,10,14,1,1,130.00
Heinrich Klaasen †,b Harshit Rana,16,17,32,1,0,94.11
Shahbaz Ahmed,c Narine b Varun,8,7,5,0,1,114.28
Abdul Samad,c †Rahmanullah Gurbaz b Russell,4,4,3,0,0,100.00
Pat Cummins (c),c Starc b Russell,24,19,28,2,1,126.31
Jaydev Unadkat,lbw b Narine,4,11,16,0,0,36.36


Score:


Score,Overs,Run Rate,Minutes
113,18.3,6.10,-


Did Not Bat:


...

Did Not Bat:


Batsman
Rinku Singh
Andre Russell
Ramandeep Singh
Mitchell Starc
Vaibhav Arora
Harshit Rana
Varun Chakravarthy


Bowlers:


Bowler,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,Wides,No Balls
Bhuvneshwar Kumar,2,0,25,0,12.50,5,2,2,1,0
Pat Cummins,2,0,18,1,9.00,7,2,1,1,0
T Natarajan,2,0,29,0,14.50,2,4,1,2,0
Shahbaz Ahmed,2.3,0,22,1,8.80,7,1,2,0,0
Jaydev Unadkat,1,0,9,0,9.00,1,1,0,0,0
Aiden Markram,1,0,5,0,5.00,2,0,0,0,0


Winner:  Kolkata Knight Riders

Man of the Match:  Mitchell Starc


In [ ]:
class Player:
    def __init__(self,player_name):
        self.player_name = player_name